In [ ]:
import os, glob, random, numpy as np, tensorflow as tf
from tensorflow.keras import layers, models

# ======= 하이퍼파라미터 =======
IMG_SIZE     = 150
NUM_CLASSES  = 68                 # [초성19 | 중성21 | 종성28]
BATCH_SIZE   = 128
EPOCHS       = 300
INIT_LR      = 3e-4
TRAIN_RATIO  = 0.8
ALPHA        = 0.75               # MobileNetV2 width multiplier
AUG_ON       = True               # 강한 증강 on/off
MODE         = "stem"             # "stem" (권장) | "native"
OUT_DIR      = "./runs_MobileNetV2"
SHARD_DIR    = r"/home/voxfish_ljy$/virtual/ValData"   # <<< 여기를 변경

AUTOTUNE     = tf.data.AUTOTUNE

# ======= L/V/T 슬라이스 인덱스 =======
L_SLICE = slice(0, 19)
V_SLICE = slice(19, 40)
T_SLICE = slice(40, 68)

# ---------------------------------------------------------
# 증강
# ---------------------------------------------------------
def hangul_ocr_augment():
    return tf.keras.Sequential([
        layers.RandomRotation(0.2),
        layers.RandomTranslation(0.15, 0.15),
        layers.RandomZoom(0.20),
        layers.RandomHeight(0.10),
        layers.RandomWidth(0.10),
    ], name="hangul_aug")

# ---------------------------------------------------------
# 모델
# ---------------------------------------------------------
def build_mobilenetv2(mode="stem", alpha=0.75, dropout=0.25, aug=True):
    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1), name="image")
    x = layers.Rescaling(2.0, offset=-1.0, name="to_minus1_1")(inp)
    if aug:
        x = hangul_ocr_augment()(x)

    if mode == "native":
        base = tf.keras.applications.MobileNetV2(
            input_shape=(IMG_SIZE, IMG_SIZE, 1),
            include_top=False, weights=None, alpha=alpha, pooling="avg",
        )
        h = base(x)
    else:
        # 1채널 -> 3채널 투영(호환성↑)
        x3 = layers.Conv2D(3, 1, padding="same", use_bias=False, name="stem_1x1")(x)
        base = tf.keras.applications.MobileNetV2(
            input_tensor=x3, include_top=False, weights=None, alpha=alpha, pooling="avg",
        )
        h = base.output

    h = layers.Dropout(dropout, name="head_dropout")(h)
    out = layers.Dense(NUM_CLASSES, activation="softmax", name="char68")(h)
    return models.Model(inp, out, name=f"mobilenetv2_{mode}")

# ---------------------------------------------------------
# 커스텀 메트릭(학습 로그에 바로 표기)
# ---------------------------------------------------------
def acc_first(y_true, y_pred):
    return tf.reduce_mean(
        tf.cast(
            tf.equal(tf.argmax(y_true[:, L_SLICE], -1), tf.argmax(y_pred[:, L_SLICE], -1)),
            tf.float32
        )
    )

def acc_middle(y_true, y_pred):
    return tf.reduce_mean(
        tf.cast(
            tf.equal(tf.argmax(y_true[:, V_SLICE], -1), tf.argmax(y_pred[:, V_SLICE], -1)),
            tf.float32
        )
    )

def acc_last(y_true, y_pred):
    return tf.reduce_mean(
        tf.cast(
            tf.equal(tf.argmax(y_true[:, T_SLICE], -1), tf.argmax(y_pred[:, T_SLICE], -1)),
            tf.float32
        )
    )

def acc_joint(y_true, y_pred):
    l_ok = tf.equal(tf.argmax(y_true[:, L_SLICE], -1), tf.argmax(y_pred[:, L_SLICE], -1))
    v_ok = tf.equal(tf.argmax(y_true[:, V_SLICE], -1), tf.argmax(y_pred[:, V_SLICE], -1))
    t_ok = tf.equal(tf.argmax(y_true[:, T_SLICE], -1), tf.argmax(y_pred[:, T_SLICE], -1))
    return tf.reduce_mean(tf.cast(l_ok & v_ok & t_ok, tf.float32))

# ---------------------------------------------------------
# 샤드 → tf.data
# ---------------------------------------------------------
def _per_shard_dataset(x_path, y_path, shuffle_inside=True):
    # 메모리는 memmap 유지 (RAM 폭증 방지)
    X = np.load(x_path, mmap_mode="r")   # (N,150,150,1) uint8
    Y = np.load(y_path, mmap_mode="r")   # (N,68) float32
    n = X.shape[0]

    # 인덱스 스트림을 만들고, 인덱스로 memmap에 접근
    ds = tf.data.Dataset.from_tensor_slices(tf.range(n, dtype=tf.int64))
    if shuffle_inside:
        ds = ds.shuffle(8192)

    def _load_one_py(i):
        i = int(i)
        return X[i], Y[i]   # numpy 배열 반환 (uint8, float32)

    def _load_one(i):
        xi, yi = tf.py_function(
            _load_one_py, [i], [tf.uint8, tf.float32]
        )
        # 정적 shape 명시 (그래프 추론 도움)
        xi.set_shape((IMG_SIZE, IMG_SIZE, 1))
        yi.set_shape((NUM_CLASSES,))
        # 전처리
        x = tf.cast(xi, tf.float32) / 255.0
        y = yi
        return x, y

    ds = ds.map(_load_one, num_parallel_calls=AUTOTUNE)
    return ds

def build_ds_from_numpy_shards(
    shard_dir: str,
    batch_size: int = 256,
    train_ratio: float = 0.9,
    interleave_across_shards: bool = True,
    repeat: bool = False,
    shuffle_files: bool = True,
):
    X_files = sorted(glob.glob(os.path.join(shard_dir, "X_shard_*.npy")))
    Y_files = sorted(glob.glob(os.path.join(shard_dir, "Y_shard_*.npy")))
    assert len(X_files) == len(Y_files) and len(X_files) > 0, "샤드 파일이 없거나 개수가 불일치합니다."

    idxs = list(range(len(X_files)))
    if shuffle_files:
        random.shuffle(idxs)

    split = int(len(idxs) * train_ratio)
    train_idx, val_idx = idxs[:split], idxs[split:]

    def make_combo(idxs, shuffle_inside=True):
        shards = [_per_shard_dataset(X_files[i], Y_files[i], shuffle_inside=shuffle_inside) for i in idxs]
        if interleave_across_shards and len(shards) > 1:
            ds = tf.data.Dataset.sample_from_datasets(shards)
        else:
            ds = shards[0]
            for s in shards[1:]:
                ds = ds.concatenate(s)
        if repeat:
            ds = ds.repeat()
        ds = ds.batch(batch_size).prefetch(AUTOTUNE)
        return ds

    train_ds = make_combo(train_idx, shuffle_inside=True)
    val_ds   = make_combo(val_idx, shuffle_inside=False) if val_idx else None
    return train_ds, val_ds, len(train_idx), len(val_idx)

# ---------------------------------------------------------
# 오프라인 평가 (원하는 시점에 호출)
# ---------------------------------------------------------
def eval_lvt_metrics(model, dataset, max_batches=None):
    L_right = V_right = T_right = Joint_right = 0
    total = 0
    for b, (X, Y) in enumerate(dataset):
        P = model.predict(X, verbose=0)  # (B,68)

        yL = np.argmax(Y[:, L_SLICE], axis=1)
        yV = np.argmax(Y[:, V_SLICE], axis=1)
        yT = np.argmax(Y[:, T_SLICE], axis=1)

        pL = np.argmax(P[:, L_SLICE], axis=1)
        pV = np.argmax(P[:, V_SLICE], axis=1)
        pT = np.argmax(P[:, T_SLICE], axis=1)

        L_right += np.sum(yL == pL)
        V_right += np.sum(yV == pV)
        T_right += np.sum(yT == pT)
        Joint_right += np.sum((yL == pL) & (yV == pV) & (yT == pT))
        total += X.shape[0]

        if max_batches is not None and (b + 1) >= max_batches:
            break

    return {
        "acc_first": L_right / total,
        "acc_mid": V_right / total,
        "acc_last": T_right / total,
        "acc_joint": Joint_right / total,
        "samples": total,
    }

# ---------------------------------------------------------
# 학습 루틴
# ---------------------------------------------------------
def compile_and_train():
    os.makedirs(OUT_DIR, exist_ok=True)

    # 1) 데이터
    train_ds, val_ds, n_tr, n_va = build_ds_from_numpy_shards(
        shard_dir=SHARD_DIR,
        batch_size=BATCH_SIZE,
        train_ratio=TRAIN_RATIO,
        interleave_across_shards=True,
        repeat=False,
        shuffle_files=True,
    )
    print(f"train shards: {n_tr}, val shards: {n_va}")

    def hangul_loss(y_true, logits):
        cho = tf.nn.softmax_cross_entropy_with_logits(
            labels=y_true[:, L_SLICE],
            logits = logits[:, L_SLICE]
        )
        jung = tf.nn.softmax_cross_entropy_with_logits(
            labels=y_true[:, V_SLICE],
            logits = logits[:, V_SLICE]
        )
        jong = tf.nn.softmax_cross_entropy_with_logits(
            labels=y_true[:, T_SLICE],
            logits = logits[:, T_SLICE]
        )
        return cho+jung+jong
    # 2) 모델
    model = build_mobilenetv2(mode=MODE, alpha=ALPHA, dropout=0.25, aug=AUG_ON)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(INIT_LR),
        loss=hangul_loss,
        metrics=[acc_first, acc_middle, acc_last, acc_joint],
    )
    model.summary()

    # 3) 콜백
    ckpt_path = os.path.join(OUT_DIR, "ckpt_best.keras")
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            ckpt_path, monitor="val_accuracy",
            save_best_only=True, save_weights_only=False
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=5, min_lr=3e-6, verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True, verbose=1
        ),
        tf.keras.callbacks.CSVLogger(os.path.join(OUT_DIR, "train.csv")),
        tf.keras.callbacks.TensorBoard(log_dir=os.path.join(OUT_DIR, "tb")),
    ]

    # 4) 학습
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1,
    )

    # 5) 최종 저장
    model.save(os.path.join(OUT_DIR, "final.keras"))

    # 6) 검증 세트에서 L/V/T/Joint 지표 출력
    if val_ds is not None:
        metrics = eval_lvt_metrics(model, val_ds)
        print("\n========== L/V/T/Joint metrics on VAL ==========")
        print(f"Samples   : {metrics['samples']}")
        print(f"초성 정확도 : {metrics['acc_first']:.4f}")
        print(f"중성 정확도 : {metrics['acc_middle']:.4f}")
        print(f"종성 정확도 : {metrics['acc_last']:.4f}")
        print(f"총 정확도   : {metrics['acc_joint']:.4f}")
        print("===============================================\n")

    return model, history, (train_ds, val_ds)

# ---------------------------------------------------------
# 실행
# ---------------------------------------------------------
if __name__ == "__main__":
    # (선택) 혼합정밀도: 지원 GPU면 주석 해제
    tf.keras.mixed_precision.set_global_policy("mixed_float16")
    compile_and_train()

train shards: 17, val shards: 5


Model: "mobilenetv2_stem"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 150, 150,  │          0 │ -                 │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ to_minus1_1         │ (None, 150, 150,  │          0 │ image[0][0]       │
│ (Rescaling)         │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hangul_aug          │ (None, None,      │          0 │ to_minus1_1[0][0] │
│ (Sequential)        │ None, 1)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_1x1 (Conv2D)   │ (None, None,      │          3 │ hangul_aug[0][0]  │
│                     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, None,      │        648 │ stem_1x1[0][0]    │
│                     │ None, 24)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, None,      │         96 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ None, 24)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, None,      │          0 │ bn_Conv1[0][0]    │
│                     │ None, 24)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, None,      │        216 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ None, 24)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, None,      │         96 │ expanded_conv_de… │
│ (BatchNormalizatio… │ None, 24)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, None,      │          0 │ expanded_conv_de… │
│ (ReLU)              │ None, 24)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, None,      │        384 │ expanded_conv_de… │
│ (Conv2D)            │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, None,      │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ None, 16)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, None,      │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, None,      │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, None,      │          0 │ block_1_expand_B… │
│ (ReLU)              │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, None,      │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ None, 96)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, None,      │        864 │ block_1_pad[0][0

 Total params: 1,469,175 (5.60 MB)

 Trainable params: 1,442,535 (5.50 MB)

...

2025-11-05 16:26:30.271229: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


3985/3985 ━━━━━━━━━━━━━━━━━━━━ 1666s 418ms/step - acc_first: 0.9767 - acc_joint: 0.9421 - acc_last: 0.9818 - acc_middle: 0.9783 - accuracy: 0.3404 - loss: 3.5331 - val_acc_first: 0.9388 - val_acc_joint: 0.8356 - val_acc_last: 0.9519 - val_acc_middle: 0.9092 - val_accuracy: 0.3279 - val_loss: 4.0390 - learning_rate: 3.0000e-04
Epoch 6/300
3985/3985 ━━━━━━━━━━━━━━━━━━━━ 1658s 416ms/step - acc_first: 0.9793 - acc_joint: 0.9487 - acc_last: 0.9838 - acc_middle: 0.9810 - accuracy: 0.3413 - loss: 3.5023 - val_acc_first: 0.2193 - val_acc_joint: 0.0152 - val_acc_last: 0.1513 - val_acc_middle: 0.2643 - val_accuracy: 0.1207 - val_loss: 20.4377 - learning_rate: 3.0000e-04
Epoch 7/300
3985/3985 ━━━━━━━━━━━━━━━━━━━━ 1668s 419ms/step - acc_first: 0.9810 - acc_joint: 0.9534 - acc_last: 0.9853 - acc_middle: 0.9829 - accuracy: 0.3428 - loss: 3.4825 - val_acc_first: 0.9332 - val_acc_joint: 0.8256 - val_acc_last: 0.9213 - val_acc_middle: 0.9253 - val_accuracy: 0.2713 - val_loss: 5.9113 - learning_rate: 3.

2025-11-05 18:18:08.110124: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


3985/3985 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - acc_first: 0.9841 - acc_joint: 0.9600 - acc_last: 0.9877 - acc_middle: 0.9854 - accuracy: 0.3430 - loss: 3.4568
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
3985/3985 ━━━━━━━━━━━━━━━━━━━━ 1698s 426ms/step - acc_first: 0.9842 - acc_joint: 0.9607 - acc_last: 0.9877 - acc_middle: 0.9855 - accuracy: 0.3435 - loss: 3.4533 - val_acc_first: 0.1057 - val_acc_joint: 0.0022 - val_acc_last: 0.3361 - val_acc_middle: 0.1461 - val_accuracy: 0.0014 - val_loss: 40.7322 - learning_rate: 3.0000e-04
Epoch 10/300
3985/3985 ━━━━━━━━━━━━━━━━━━━━ 1692s 424ms/step - acc_first: 0.9878 - acc_joint: 0.9699 - acc_last: 0.9902 - acc_middle: 0.9894 - accuracy: 0.3454 - loss: 3.4130 - val_acc_first: 0.4699 - val_acc_joint: 0.1187 - val_acc_last: 0.8104 - val_acc_middle: 0.3677 - val_accuracy: 0.0605 - val_loss: 28.5079 - learning_rate: 1.5000e-04
Epoch 11/300
3985/3985 ━━━━━━━━━━━━━━━━━━━━ 1686s 423ms/step - acc_first: 0.9887 - acc_joint: 

KeyError: 'acc_middle'